In [1]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig

In [2]:
model_loaded = BertForSequenceClassification.from_pretrained("/Users/jakar/Downloads/model/mm")
tokenizer = BertTokenizer.from_pretrained("/Users/jakar/Downloads/model/mm")
sent = "i is a student"
encoded_dict = tokenizer.encode_plus(
        sent,  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=64,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
    )
input_id = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']
input_id = torch.LongTensor(input_id)
attention_mask = torch.LongTensor(attention_mask)
with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model_loaded(input_id, attention_mask=attention_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\jakar\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
logits = outputs[0]
index = logits.argmax()
torch_label = index.detach().cpu().numpy()
print(torch_label)

0


In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [4]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("/Users/jakar/Downloads/model/bert.onnx")

In [7]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_id),
                        ort_session.get_inputs()[1].name: to_numpy(attention_mask)}

In [10]:
import torch.nn.functional as F
ort_outs = ort_session.run(["output"], ort_inputs)
torch_onnx_output = torch.tensor(ort_outs[0], dtype=torch.float32)
onnx_logits = F.softmax(torch_onnx_output, dim=1)
logits_label = torch.argmax(onnx_logits, dim=1)
label = logits_label.detach().cpu().numpy()
print(label)

[0]
